In [ ]:
import cloudscraper
from bs4 import BeautifulSoup
import math
import pprint
import time
import pickle

# Takes a public Discogs store inventory and returns pricing information on other listings on the market.


# Classes

class formattedListings: # Formatted marketplace listings for a single release.
    
    def __init__(self,title,url,listings,place,total):
        self.title = title
        self.url = url
        self.listings = listings
        self.place = place
        self.total = "Total: {0}".format(total)

    def __str__(self):
        return "{0}\n{1}\n{2}\n{3}".format(self.title,self.url,self.listings,self.total)

    
# Helper Functions

def get_inventory(username): # Given a seller username, gets their store url and inventory count.

    URL = "https://www.discogs.com/seller/{0}/profile".format(username)
    pages = count_pages(URL)
    
    return parse_list(URL, pages)

def count_pages(URL): # Takes URL for a Discogs store, returns the number of pages.
    
    html = scraper.get(URL).content
    soup = BeautifulSoup(html, 'html.parser')
    
    inventory_size = int(soup.find(id="page_content").find("li", class_="first").find("h2").text.strip().strip("For Sale"))
    pages = math.ceil(inventory_size/250)
            
    return pages    

def parse_list(URL, pages): # Takes URL of a store inventory, returns a list of the releases and their item ids.
    
    new_list = []

    for page in range(1, pages + 1):
        new_URL = URL + "?&limit=250&sort=artist&sort_order=asc&page={0}".format(page)
        html = scraper.get(new_URL).content
        soup = BeautifulSoup(html, 'html.parser')

        list_items = soup.find(id="pjax_container").find("tbody").find_all("tr")
        for item in list_items:
            release = item.find("td", class_="item_description")
            title = release.find("strong").text.strip()
            item_id = release.find("a", class_="item_release_link")["href"].split("-")[0].strip("/release/")

            new_list_item = (title, item_id)
            new_list.append(new_list_item)
    
    return new_list

def get_listings(username, release_title, item_id): # Given username and item_id, scrapes marketplace for listings and stores them in provided list.
    
    URL = "https://www.discogs.com/sell/release/{0}?ships_from=United+States&sort=price%2Casc".format(item_id)
    html = scraper.get(URL).content
    soup = BeautifulSoup(html, 'html.parser')
    
    count = 0
    your_place = 0
    formatted_listings = ""
    
    listings = soup.find("table", class_="mpitems").find_all("tr", class_="shortcut_navigable")
    for listing in listings:
        count += 1
        if is_user(username, listing):
            formatted_listings += "{0} (You) ({1})\n".format(get_price(listing), count)
            your_place = count
        else:
            if check_scam(listing):
                formatted_listings += "{0} (SCAM)\n".format(get_price(listing))
            else:
                formatted_listings += "{0}\n".format(get_price(listing))
                
    total = (soup.find("strong", class_="pagination_total").text.split(" of "))[-1]
    entry = formattedListings(release_title,URL,formatted_listings,your_place,total)
    
    if your_place < 10:
        (sorted_inventory_list[your_place - 1]).append(entry)
    else:
        sorted_inventory_list[9].append(entry)
    inventory_list.append(entry)
                        
    return

def is_user(username, listing): # Checks if a marketplace listing matches the provided username.
    
    return listing.find(string=username)

def check_scam(listing): # Checks if a listing is a scam (has 0.0% seller rating).
    
    return listing.find(string="0.0%")

def get_price(listing): # Gets price provided a listing.

    try:
        item_condition = listing.find("p", class_="item_condition").text
        formatted_condition = format_condition(item_condition)
        
        if listing.find(string="New seller"):
            return "{0} {1} (New)".format(listing.find("span", class_="converted_price").text.strip(), formatted_condition)
        else:
            return "{0} {1}".format(listing.find("span", class_="converted_price").text.strip(), formatted_condition)
    except AttributeError:
        return "n/a"

def format_condition(item_condition): # Formats the item condition to (Media/Sleeve).
    
    media = (item_condition.split("("))[1].split(")")[0]
    try:
        sleeve = (item_condition.split("("))[2].split(")")[0]
    except IndexError:
        return "({0})".format(media.split(" or")[0])

    media = media.split(" or")[0]
    sleeve = sleeve.split(" or")[0]
    
    return "({0}/{1})".format(media, sleeve)

def print_sorted_list(sorted_inventory_list): # Given a sorted inventory list, prints it out.
    
    count = 0
    for place in sorted_inventory_list:
        
        counter = 0        
        for entry in place:
            count += 1
            counter += 1
            print("({0})".format(count))
            print("{0}\n".format(entry))
            
        places.append(counter)
        print("Count: {0}".format(counter))
        print('─' * 25)
        
    print("Place")
    for place in range(len(places)):
        print("{0}: {1}".format(place+1, places[place]))
    print("\n")
        
def print_list(unsorted_inventory_list): # Prints unsorted inventory list.
    
    count = 0       
    for entry in unsorted_inventory_list:
        count += 1
        print("({0})".format(count))
        print("{0}\n".format(entry))
        
def compare_inventory_list(inventory_list): # Compares inventory list with pickled list for changes.
    
    pickled_list = load_state()
    
    if pickled_list:
        print("Loaded saved inventory state.\n")
        
        if len(pickled_list) == len(inventory_list):
            for count in range(len(pickled_list)):
                pickled_entry = pickled_list[count]
                compare_entries(pickled_entry, count)
        else:
            print("Inventory size changed.")
        print("Finished comparison.\n")
    else:
        print("Nothing to load.\n")
        print_list(inventory_list)

    # save_state(inventory_list)
    # print("Saved inventory list as pickle.")
    
def compare_entries(pickled_entry, count): # Given an unpickled entry and an entry number, compares that pickled entry to the current inventory list.
    
    listings = inventory_list[count].listings
    pickled_listings = pickled_entry.listings
    current_place = inventory_list[count].place
    old_place = pickled_entry.place
    
    if listings != pickled_listings:
        print("({0})".format(count))
        print("{0}\n{1}".format(pickled_entry.title, pickled_entry.url))
        
        compare_listings(listings, pickled_listings)
        
        if current_place != old_place:
            print("Place: {0} --> {1}\n".format(old_place, current_place))
            
def compare_listings(current_listings, pickled_listings): # Given a list of listings, compares the current and pickled versions.
    
    current_list = current_listings.split("\n")
    pickled_list = pickled_listings.split("\n")
    
    for index in range(len(current_list)):
        try:
            current_listing = current_list[index]
            pickled_listing = pickled_list[index]

            if current_listing == pickled_listing:
                print(current_listing)
            else:
                if pickled_list[index+1] == current_list[index] or pickled_list[index+1].split("(You)")[0] == current_list[index].split("(You)")[0]:
                    print("{0} --> {1}".format(pickled_listing, "Removed"))
                    current_list.insert(index,pickled_listing)
                elif current_list[index+1] == pickled_list[index] or current_list[index+1].split("(You)")[0] == pickled_list[index].split("(You)")[0]:
                    print("{0} --> {1}".format("Inserted", current_listing))
                    pickled_list.insert(index,current_listing)
                elif ("You" in pickled_listing) and ("You" in current_listing):
                    print("{0} --> {1}".format(pickled_listing, current_listing.split("(You) ")[-1]))
                else:
                    print("{0} --> {1}".format(pickled_listing, current_listing))
                
        except IndexError:
            print("Length of listings does not matched to saved listings.")
    print("\n")

def save_state(inventory_list): # Pickles an inventory list as a save state in a bin file.
    
    with open("state.bin", "wb") as f:
        pickle.dump(inventory_list, f)
    
def load_state(): # Loads a pickled inventory list from a bin file.
    
    with open("state.bin", "rb") as f:
        try:
            pickled_list = pickle.load(f)
            return pickled_list
        except pickle.UnpicklingError:
            return []
        

In [332]:
# Main

scraper = cloudscraper.create_scraper(browser={'browser': 'firefox','platform': 'windows','mobile': False})

#print("Enter username: \n")
#user1 = input()
user1 = "curefortheitch"

try:
    start_time = time.time()
    inventory = get_inventory(user1)

    inventory_list = []
    sorted_inventory_list = [[],[],[],[],[],[],[],[],[],[]]
    places = []
    
    print("Loading inventory...\n")
    for release in inventory:
        get_listings(user1,release[0],release[1])
     
    choice = "0"
    while choice != "5":
        print('─' * 50)
        print("Choose option:\n(1) Print sorted list.\n(2) Print unsorted list.\n(3) Check for changes.\n(4) Save inventory list.\n(5) Quit.\n")
        choice = input()
        print('─' * 50)
        if choice == "1":
            print_sorted_list(sorted_inventory_list)
        elif choice == "2":
            print_list(inventory_list)
        elif choice == "3":
            compare_inventory_list(inventory_list)
        elif choice == "4":
            save_state(inventory_list)
            print("Saved inventory list as pickle.")
    
    # print_sorted_list(sorted_inventory_list)
    # print_list(inventory_list)
    # compare_inventory_list(inventory_list)
    
    print("\nDone.")
    end_time = time.time()
    print("Total time: {0} seconds".format(end_time-start_time))
        
except AttributeError:
    print("Can't find user.")


Loading inventory...

──────────────────────────────────────────────────
Choose option:
(1) Print sorted list.
(2) Print unsorted list.
(3) Check for changes.
(4) Save inventory list.
(5) Quit.

1
──────────────────────────────────────────────────
(1)
Blackbear - Everything Means Nothing (LP, Album, Ltd, Bab)
https://www.discogs.com/sell/release/15910680?ships_from=United+States&sort=price%2Casc
$58.99 total (VG+/VG+) (You) (1)
$64.00 total (M/VG+)
$67.20 total (M/M)
$64.00 total (M/VG+)
$75.00 total (M/M) (New)
$75.00 total (NM/NM)
$80.00 total (M/M)
$81.00 total (M/M)
$76.99 total (NM/NM)
$85.00 total (M/M)
$92.50 total (M/M)
$94.98 total (M/M)
$99.34 total (M/NM)
$104.97 total (M/M)
$104.50 total (M/M)
$109.50 total (M/M)
$109.50 total (M/M)
$124.50 total (M/M)
$145.00 total (M/M) (New)
$150.00 total (M/NM)

Total: 20    

(2)
blackbear - Digital Druglord (LP, RE, Ora)
https://www.discogs.com/sell/release/24535436?ships_from=United+States&sort=price%2Casc
$75.00 total (M/NM) (You) (

3
──────────────────────────────────────────────────
Loaded pickled list.

(1)
Action Bronson - White Bronco (LP, Album, Ltd, Tou)
https://www.discogs.com/sell/release/15973952?ships_from=United+States&sort=price%2Casc
Inserted --> $92.30 total (VG+/NM) (New)
n/a
$166.00 total (NM/NM) (New)
$192.80 total (NM/NM)
$205.00 total (M/NM)
$228.45 total (M/M)
$256.00 total (NM/NM)
$305.00 total (NM/NM) (You) (7) --> (8)
$306.50 total (M/NM)
$330.00 total (M/M) (SCAM)
$500.00 total (M/NM) (New)
$701.96 total (NM/NM)



Place: 7 --> 8

(5)
A$AP Rocky* - At.Long.Last.A$AP (2xLP, Album, Ltd, Whi)
https://www.discogs.com/sell/release/8967794?ships_from=United+States&sort=price%2Casc
Inserted --> $98.74 total (NM/NM) (New)
$133.00 total (NM/VG+)
$155.99 total (VG+/VG+)
$156.00 total (M/M)
$195.00 total (M/VG+)
$205.00 total (M/NM) (You) (5) --> (6)



Place: 5 --> 6

(6)
Avicii - Stories (2xLP, Album, Gat)
https://www.discogs.com/sell/release/15657723?ships_from=United+States&sort=price%2Casc
Inser